## Importing relevant libraries

In [1]:
import numpy as np
import scipy.linalg as la
import scipy.spatial as spat
from scipy.stats import unitary_group
from scipy.stats import moment
from scipy.stats import skew, kurtosis
from scipy.optimize import curve_fit
from scipy.linalg import norm
import matplotlib.pyplot as plt
import math
from dataclasses import dataclass
import time
from datetime import datetime

# Libraries for implementing the VQD algorithm
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector, Gate
from qiskit.primitives import Sampler, Estimator
from qiskit_aer import AerSimulator
from qiskit_algorithms.utils import algorithm_globals
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session, Options
from qiskit_ibm_runtime import Estimator as EstimatorNew
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import RealAmplitudes, TwoLocal, EfficientSU2
from qiskit_algorithms.optimizers import *
from qiskit_algorithms.state_fidelities import ComputeUncompute

from qiskit_algorithms.eigensolvers import EigensolverResult, VQD
from qiskit_algorithms import NumPyMinimumEigensolver, VQE

# Import classical optimizers
from qiskit_algorithms.optimizers import SPSA, P_BFGS, COBYLA, IMFIL, SNOBFIT, NELDER_MEAD, SLSQP, NFT, ADAM, POWELL, GradientDescent, BOBYQA

# Import Statevector and SparsePauliOp
from qiskit.quantum_info import SparsePauliOp, Statevector

# Import noise models
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

# Import a fake backend and Qiskit simulators and/or noise libraries
from qiskit_aer import AerSimulator
# from qiskit_aer.primitives import Estimator as AerEstimator 
# from qiskit_aer.noise import NoiseModel

# Import the FakeManila backend
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeMontrealV2, FakeGuadalupeV2, FakeManila

## Constructing the FPE operator

In [3]:
## Classical implementation
"""
Purpose: 
    Find the analytical solution given the input parameters
Input: 
    x: array representing the x coordinates over which the solution is to be found
    a, Γ: parameters for the Ornstein-Uhlenbeck equation
Output:
    y: array of the value of the analytical solution over x
"""

def OrnsteinUhlenbeck(x, a = 1, gamma = 1):
    y = np.sqrt(a/(2*np.pi*gamma)) * np.exp((-a * x ** 2)/(2*gamma))
    
    return y

"""
Purpose:
    Calculate the weighted Hermite polynomials using recursive relations up to H_n
Input:
    n: the highest number of Hermite polynomials to be evaluated at
    x: the value of x at which the weighted Hermite polynomials are to be evaluated
Output:
    Hermite: the value of the weighted Hermite polynomials
Example: HermiteSeries(2, 4) -> [H_0(4), H_1(4), H_2(4)]
"""

def HermiteSeries(n, x):
    Hermite = np.zeros((1, n+1))
    Hermite[0][0] = 1.0 * np.exp(-x**2/2)
    Hermite[0][1] = 2.0 * x *np.exp(-x**2/2)
    
    for ni in range(1, n):
        Hermite[0][ni+1] = 2*x*Hermite[0][ni] - 2*ni*Hermite[0][ni-1]
    
    return Hermite

"""
Purpose:
    Reconstruct the functions represented by the coefficients of weighted Hermite polynomials in spatial coordinates
Input:
    x: array containing the x values over which the function is to be evaluated
    coefficients: the coefficients corresponding to the Hermite polynomials (should be zero-mode of the operator)
    nmax: the highest order of weighted Hermite polynomials used
    s: the scaling factor
Output:
    y: the reconstruction
"""

def project(x, coefficients, nmax, s):
    
    y = np.zeros(x.shape)
    
    for i in (range(len(x))):
        x0 = s*x[i]
        hermite_values = HermiteSeries(nmax, x0)
        y[i] += np.dot(hermite_values, coefficients)
        
    return (x, y)

"""
Purpose:
    Normalize the PDF so that the area under the curve is 1
Input:
    x, y: arrays representing the unnormalized
Output:
    (x, y): normalized PDF
"""

def normalize(x, y):
    dx = x[1] - x[0]
    sum = 0
    
    for i in range(len(x)):
        sum = sum + (y[i] * dx)
    
    y = y/sum
    
    return (x, y)

"""
Purpose:
    Calculate the RMS
Input:
    y_op, y: the y values generated by the operator and the analytical solution
Output:
    RMS
"""

def rms_diff(y_op, y):
    return np.sqrt(np.sum((y_op-y)**2)/len(y))

"""
Purpose:
    Find s given a and Γ such that the projection in Hermite space is just H_0
Input:
    a, Γ: parameters for Ornstein-Uhlenbeck
Output:
    s
"""

def special_s(a, gamma):
    return np.sqrt(a/gamma)

def euler_representation(z):
    A = abs(z)
    phase = math.atan(z.imag/z.real)
    
    return A, phase

def generate_positive_semidefinite_matrix(eigenvalues):
    ## Function to transform the FPE operator to the Hermite basis and return a finite truncated matrix for the ...
    ## ... FPE operator
    
    n = len(eigenvalues)
    s = np.diag(eigenvalues)
    q, _ = la.qr(np.random.rand(n, n))
    semidef = np.dot(np.transpose(q), s)
    semidef = np.dot(semidef, q)
    
    return semidef

def normalize_probability(y, dx):
    ## Function to return a normalized PDF 
    
    total = np.sum(y) * dx
    y = y / total
    
    return y

def perturbed_ornstein_uhlenbeck(x0, a = 1, gamma = 1, c = 0, shift = 0):
    ## Function to compute the exact solution to the perturbed Ornstein-Uhlenbeck equation
    
    assert len(x0) > 1, "Error: x0 should be a vector"

    x = x0 - shift
    y = np.exp(-a*x**2 /(2*gamma) - c*x**4 /(4*gamma))
    dx = x[2] - x[1]
    
    y = normalize_probability(y, dx)
    
    return y

def special_L(a, gamma):
    ## Function to compute the characteristic length scale
    
    return np.sqrt(a/gamma)

def delta(N, n):
    ## Function to implement the Kronecker delta function
    
    if N == n:
        return 1
    else:
        return 0
    
def matrix_element(N, n, L):
    # Function to compute the matrix elements of the position operator
    
    value = L/np.sqrt(2) * (np.sqrt(n) * delta(N, n-1) + np.sqrt(n+1) * delta(N, n+1))
    
    return value

def create_position(nmax, L):
    # Function to construct the position operator
    
    op = np.zeros((nmax + 1, nmax + 1))
    
    for N in range(nmax+1):
        row = N
        for n in range(nmax+1):
            col = n
            op[row, col] = matrix_element(N, n, L)
            
    return op

def f(N,n):
    return 0.5*( np.sqrt(n*(n-1))*delta(N,n-2) + delta(N,n) - np.sqrt((n+1)*(n+2))*delta(N,n+2))

def g(N,n, L):
    return (0.5 / L**2)*( np.sqrt(n*(n-1))*delta(N,n-2) - (2*n+1)*delta(N,n) + np.sqrt((n+1)*(n+2))*delta(N,n+2))

def t(N, n, L):
    val1 = np.sqrt(n*(n-1)*(n-2)*(n-3))*delta(N, n-4)
    val2 = (2*n+2)*np.sqrt(n*(n-1))*delta(N, n-2)
    val3 = (6*n+3)*delta(N, n)
    val4 = -2*n*np.sqrt((n+1)*(n+2))*delta(N, n+2)
    val5 = -1*np.sqrt((n+1)*(n+2)*(n+3)*(n+4))*delta(N, n+4)

    return (L**2/4)*( val1 + val2 + val3 + val4 + val5 )

def element_perturbed(N, n, L, a, c, gamma):
    ## Function to compute the matrix elements of the finite truncated matrix for the FPE operator (in the Hermite basis)
    
    return -(a*f(N,n) + c*t(N, n, L) + gamma*g(N, n, L))

def create_operator_perturbed(nmax, L, a, c, gamma):
    ## Function to generate the FPE operator matrix in the Hermite basis
    op = np.zeros((nmax+1, nmax+1))
    
    for N in range(nmax+1):
        row = N
        for n in range(nmax+1):
            col = n
            op[row, col] = element_perturbed(N, n, L, a, c, gamma)
    
    """
    if np.any(np.nan, op):
        print("There's an nan in the operator")
    elif np.any(np.inf, op):
        print("There is an inf in the operator")
    """

    return op

def state_n(nmax, x0, L):
    ## Function to compute the Hermite polynomials upto the order specified by nmax
    
    assert L > 0, "Error (state_n): input L must be greater than or equal to 0"

    states = np.zeros(nmax + 1)
    x = x0 / L

    states[0] = np.sqrt(L / np.sqrt(1/np.pi))* 1.0 * np.exp(-x**2/2)

    if nmax > 0:
        states[1] = np.sqrt(L / np.sqrt(1/np.pi))* (1/np.sqrt(2)) * 2.0 * x * np.exp(-(x**2)/2)
    
    for ni in range(2, nmax + 1):
        states[ni] = ((np.sqrt(2)*x) / np.sqrt(ni))*states[ni-1] - (ni-1)/np.sqrt((ni)*(ni-1))*states[ni-2]

    return states

def integrate_eigenvector(x0, y, nmax, L):
    ## Function to compute the coefficients corresponding to the calculated Hermite polynomials by numerical integration
    ## Note: here, we use the left-hand integration method
    
    dx = x0[2] - x0[1]
    eigenvector = np.zeros(nmax + 1)

    for i in range(len(x)):

        states = state_n(nmax, x0[i], L)
        states = states * y[i] * dx

        eigenvector = eigenvector + states
        
    return {"eigenvector" : eigenvector, "dx" : dx, "nmax" : nmax, "x0" : x0, "L" : L}

def make_operator_even(op):
    op_new = np.zeros((op.shape[0]//2, op.shape[1]//2))
    
    for row in range(op_new.shape[0]):
        for col in range(op_new.shape[1]):
            op_new[row, col] = op[row*2, col * 2]
    
    return op_new

def reconstruct_eigenvector(cache, normalize = True, only_even = False):
    ## Function to reconstruct the PDF using the cache obtained previously
    
    eigenvector = cache["eigenvector"]
    nmax = cache["nmax"]
    x0 = cache["x0"]
    dx = cache["dx"]
    L = cache["L"]
    
    if not only_even:
        eigenvector = cache["eigenvector"]
    else:
        eigenvector_old = cache["eigenvector"]
        eigenvector = np.zeros(nmax + 1)
        
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]

    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    if normalize:   
        y = normalize_probability(y, dx)
        
    return (x0, y)

def find_zeromode(op_nonhermitian, nmax, x0, dx, L, which = "single", only_even = False):
    ## Function to compute the zeromode, its position, and the eigenvalue corresponding to the zeromode
    
    assert which == "nonhermitian" or which == "single", "Error: currently only supports which = {\"nonhermitian\", \"single\"}"
    
    if only_even:
        op_processed = make_operator_even(op_nonhermitian)
    else:
        op_processed = op_nonhermitian
        
    if which == "nonhermitian":
        op = op_processed
    elif which == "single":
        op = np.dot(np.transpose(op_processed), op_processed)

    eigenvalues, eigenvectors = la.eig(op)

    index = np.argmin(np.real(eigenvalues)**2)

    min_eigenval = eigenvalues[index]
    
    zeromode = eigenvectors[:, index]

    return {"operator" : op, "which" : which, \
            "eigenvector" : np.real(zeromode), "index" : index, "eigenvalue" : np.real(min_eigenval), \
            "nmax" : nmax, "x0" : x0, "dx" : dx, "L" : L, "spectrum" : eigenvalues, "P" : eigenvectors}

def rms_diff(y0, y, sigdigits = 0):
    ## Function to compute the RMS difference between the analytically computed PDF and the one ...
    ## ... obtained using the FPE operator
    
    assert len(y0) == len(y), "Error: the length of y0 and y should be the same"
    diff = np.sum((y0-y)**2)/len(y0)
    
    return np.sqrt(diff)

def analyze_collapse(initial_state, P, index):
    num_basis = P.shape[1]
    coefficients = np.zeros((num_basis, ))

    for i in range(num_basis):
        basis = P[:, i]
        basis = normalize_eigenvector(basis)

        coefficients[i] = np.dot(basis, initial_state)

    normalized_coefficients = normalize_eigenvector(coefficients)

    reconstruction = np.zeros((P.shape[0], ))

    for i in range(num_basis):
        basis = P[:, i]
        basis = normalize_eigenvector(basis)

        added_vector = coefficients[i] * basis
        reconstruction = reconstruction + added_vector

    probability = normalized_coefficients ** 2
    
    probability_zeromode = probability[index]
    assert np.sum(probability) - 1 < 0.00001, "Error: unity normalization of probability is not observed"

    print("The initial_state is: ")
    print(initial_state)
    print()

    print("The coefficient expansion is: ")
    print(coefficients)
    print()

    print("The normalized coefficient expansion is: ")
    print(normalized_coefficients)
    print()

    print("The reconstructed vector is: ")
    print(reconstruction)
    print()

    print("The probability of collapsing into each eigenvector is: ")
    print(probability)
    print()

    print("The probability of collapsing into the zeromode is: ")
    print(probability_zeromode)
    print()
    
    return probability_zeromode, probability

def get_unitary(matrix, add_half = False):
    """
    Purpose: given a matrix, returns the unitary, hermitian matrix to be diagonalized
    Input: matrix -> the matrix to be diagonalized
    Output: U -> the unitary matrix
            nqubits -> the number of qubis needed to represent the basis of U
            dimension -> the dimension of the original matrix
    """
    assert matrix.ndim == 2, "Error: Only a matrix maybe processed"
    assert matrix.shape[0] == matrix.shape[1], "Error: Only a square matrix maybe processed"

    if np.any(np.transpose(matrix) != matrix):
        matrix_T = np.transpose(matrix)
        matrix = np.dot(matrix_T, matrix)

    ## Finding the dimension of the matrix
    dimension_hermitian = matrix.shape[0]

    ## Finding the number of qubits required to represent the matrix
    nqubits = int(np.ceil(np.log2(dimension_hermitian)))

    ## Construct the relevant matrix
    op_dim = 2 ** nqubits
    op = np.eye(op_dim)
    op[0:dimension_hermitian, 0:dimension_hermitian] = np.copy(matrix)

    if add_half:
        op = op + np.pi * np.eye(op.shape[0])

    U = la.expm(1j*op)

    # Get the dimensions of the unitary matrix
    dimension = U.shape[0]

    return U, nqubits, dimension

def expect_value(zeromode, matrix):
    
    value = np.dot(matrix, zeromode)
    expect = np.dot(np.transpose(zeromode), value)
    
    return expect

## Helper functions

In [6]:
def find_probability(eigenvector_raw):
    """
    Purpose: Find the probability associated with each basis of an eigenvector
    Input: eigenvector_raw -> Numpy array documenting the number of times each basis is detected within the eigenvector
    Output: eigenvector_prob -> Numpy array documenting the probability of detecting each basis
    """
    count_total = np.sum(eigenvector_raw)
    eigenvector_prob = eigenvector_raw / count_total
    
    return eigenvector_prob

def find_amplitude(eigenvector_prob):
    """
    Purpose: Finding the probability amplitude of each basis using quantum mechanics
    Input: eigenvector_prob -> Numpy array documenting the probability that each basis is measured
    Output: eigenvector -> Numpy array representing the eigenvector
    """
    eigenvector = np.sqrt(eigenvector_prob)
    return eigenvector

def normalize_eigenvector(vector):
    """
    Purpose: Normalizes a vector such that its norm is 1
    Input: vector -> The vector to be normalized
    Output: vector -> The normalized vector
    """
    L2 = np.sum(np.square(vector))
    vector = vector / np.sqrt(L2)

    return vector

def make_operator_even(op):
    op_new = np.zeros((op.shape[0]//2, op.shape[1]//2))

    for row in range(op_new.shape[0]):
        for col in range(op_new.shape[1]):
            op_new[row, col] = op[row*2, col * 2]

    return op_new

def get_pdf(n, x, dx, L, shift, zeromode_qpe, normalize = True, make_even = False):
    # Function to construct the ground state PDF using the VQSVD zeromode
    
    if not make_even:
        eigenvector = zeromode_qpe
    else:
        eigenvector_old = zeromode_qpe
        eigenvector = np.zeros(n + 1)
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]
            
    x0 = x - shift
    
    # Computing the PDF
    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    
    if normalize:
        y = normalize_probability(y, dx)

    return x0, y

def compute_expectation_x_squared_simpson(x, y, n):
    """
    Computes the expectation value of x^2 using Simpson's rule for numerical integration.
    
    Parameters:
    x (array-like): Discrete values of x.
    y (array-like): Corresponding values of the probability density function (PDF) at x.
    
    Returns:
    float: The expectation value of x^2.
    """
    # Ensure x and y are numpy arrays
    x = np.array(x)
    y = np.array(y)
    
    # Compute x^2
    x_squared = x**n
    
    # Check if the number of intervals is even, if not make it even by truncating the last point
    if len(x) % 2 == 0:
        x = x[:-1]
        y = y[:-1]
        x_squared = x_squared[:-1]
    
    # Compute the integral using Simpson's rule
    h = (x[-1] - x[0]) / (len(x) - 1)
    integral = y[0] * x_squared[0] + y[-1] * x_squared[-1] + \
               4 * np.sum(y[1:-1:2] * x_squared[1:-1:2]) + \
               2 * np.sum(y[2:-2:2] * x_squared[2:-2:2])
    integral *= h / 3
    
    return integral

def get_pdf(n, x, dx, L, shift, zeromode_qpe, normalize = True, make_even = False):
    # Function to construct the ground state PDF using the VQSVD zeromode
    
    if not make_even:
        eigenvector = zeromode_qpe
    else:
        eigenvector_old = zeromode_qpe
        eigenvector = np.zeros(n + 1)
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]
            
    x0 = x - shift
    
    # Computing the PDF
    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    
    if normalize:
        y = normalize_probability(y, dx)

    return x0, y

# Fidelity measure 1
def get_fidelity(zeromode_classic, zeromode_vqe):
    # Function to compute the infidelity

    overlap = np.dot(np.transpose(zeromode_vqe), zeromode_classic)
    fidelity = 1 - overlap ** 2
    return fidelity

# Fidelity measure 2
def get_similarity(a, b):
    # Function to compute the similarity between 2 zeromodes
    
    numerator = np.abs(np.dot(a.conj().T, b))**2
    denominator = np.linalg.norm(a)**2 * np.linalg.norm(b)**2
    
    return numerator / denominator

def compute_errors(expect_classical, expect_quantum):

    error = np.abs(expect_classical - expect_quantum) / expect_classical
    return error

## VQE implementation

In [9]:
# VQE run for a given ansatz and optimizer
def run_vqe_ansatz_analysis(matrix, ansatz, optimizer, seed, exact_ground_state, classical_expectation, nmax, L):

    # Define parameters to compute <x^2>
    dx = 0.01
    x = np.linspace(-4, 4, int(8/dx))

    # Get the Pauli-decomposed form of the operator
    qub_hamiltonian = SparsePauliOp.from_operator(matrix)
    dimension = matrix.shape[0]
    num_qubits = int(np.log2(dimension))

    # Set up the random initial point
    np.random.seed(seed)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)

    # Initialize the Estimator primitive
    estimator = Estimator()

    # Logging class for VQE callback
    @dataclass
    class VQELog:
        parameters: list
        values: list
        def update(self, count, parameters, mean, _metadata):
            self.values.append(mean)
            self.parameters.append(parameters)
            
    log = VQELog([], [])
    
    # Run VQE with the given ansatz and optimizer
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point, callback=log.update)

    # Get the VQE results
    result = vqe.compute_minimum_eigenvalue(qub_hamiltonian)

    # Get the number of optimizer function calls
    num_calls = result.cost_function_evals

    # Extract the optimal parameters and construct the state vector
    optimal_params = result.optimal_point
    final_circuit = ansatz.assign_parameters(optimal_params)
    vqe_statevector = Statevector.from_instruction(final_circuit)

     # Convert the quantum and classical zeromodes into 4 x 1 arrays
    exact_ground_state = np.array(exact_ground_state).reshape((len(exact_ground_state), 1))
    vqe_statevector = vqe_statevector.data.tolist()

    if len(exact_ground_state) == 6:
        vqe_statevector = vqe_statevector[:6]
        zeromode = np.array(vqe_statevector).reshape((len(exact_ground_state), 1))
        zeromode = np.real(zeromode)
    
        # Compute the fidelity measure
        fidelity_value = get_similarity(exact_ground_state, zeromode)
    
    else:
        zeromode = np.array(vqe_statevector).reshape((len(exact_ground_state), 1))
        zeromode = np.real(zeromode)
    
        # Compute the fidelity measure
        fidelity_value = get_similarity(exact_ground_state, zeromode)

    # Compute <x^2>
    x_quantum, y_quantum = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    quantum_expectation = compute_expectation_x_squared_simpson(x_quantum, y_quantum, 2)
    error = compute_errors(classical_expectation, quantum_expectation)
        
    return zeromode, fidelity_value, error, num_calls

## Construct the matrix and the zeromode

In [12]:
def get_zeromode(nmax, a, c, L, gamma):
    # Function to construct the matrix and get the zeromode

    #3, 5, 1, 1
    dx = 0.01
    x = np.linspace(-4, 4, int((8)/dx))
    
    ## Finding the zeromode through diagonalization
    op_nonhermitian = create_operator_perturbed(nmax, L, a, c, gamma)
    only_even = True
    
    # Matrix
    cache_diagonalization = find_zeromode(op_nonhermitian, nmax, x, dx, L, which = "single", only_even = only_even)
    matrix = cache_diagonalization['operator']

    # Get the classical zeromode
    A, P = la.eig(matrix)
    
    # Get the zeromode
    A_real = np.real(A)
    index = np.where(A_real == np.amin(A_real))[0][0]
    
    eigenvalue = A[index]
    zeromode_classic = P[:, index]
    
    zeromode_classic = np.real(normalize_eigenvector(zeromode_classic))
    zeromode_classic = np.reshape(zeromode_classic, (zeromode_classic.size, 1))

    return matrix, zeromode_classic

## Estimate quantum resources

In [15]:
def estimate_resources(matrix, zeromode_classic, optimizers, num_qubits, target_error_threshold, classical_expectation, nmax, L):
    
    # Initial depth and maximum depth
    initial_depth = 1
    max_depth = 10  # Set the maximum number of depths to check

    # Initialize dictionaries to store minimum depths, zeromodes, and resources for each optimizer-ansatz pair
    min_depths = {}
    relative_errors = {}
    resource_info = {}  # To store gate count, circuit depth, and function calls
    
    # Loop through each optimizer
    for optimizer in optimizers:
        optimizer_name = optimizer.__class__.__name__
        
        # Loop through each ansatz type and configure dynamically
        for AnsatzClass in [RealAmplitudes, TwoLocal, EfficientSU2]:
            # Initialize ansatz with appropriate configurations
            if AnsatzClass == RealAmplitudes:
                ansatz = AnsatzClass(num_qubits=num_qubits, entanglement='full', reps=initial_depth)
            elif AnsatzClass == TwoLocal:
                ansatz = AnsatzClass(num_qubits=num_qubits, rotation_blocks=['ry'], entanglement_blocks='cx', reps=initial_depth)
            elif AnsatzClass == EfficientSU2:
                ansatz = AnsatzClass(num_qubits=num_qubits, su2_gates=['ry'], entanglement='sca', reps=initial_depth)
            
            ansatz_name = AnsatzClass.__name__
            pair_name = f"{optimizer_name}-{ansatz_name}"
            print(f"\nRunning VQE for optimizer-ansatz pair: {pair_name}")
            
            current_depth = initial_depth
            converged = False  # Flag to check if convergence occurs
            
            while current_depth <= max_depth:  # Loop for up to max_depth
                # Set `reps` (depth) for the current ansatz
                ansatz.reps = current_depth
                
                # Temporary storage for the relative error results
                fidelities = []
                function_calls = []  # To track the number of function calls for each run
                all_relative_errors = [] # To store all the relative errors
                
                # Perform multiple independent VQE runs to calculate average relative error
                for run in range(10):  # Number of independent runs
                    # Set a seed for a specific VQE run
                    seed = run + 1
                    
                    # Run VQE
                    zeromode, fidelity_value, error, function_call_count = run_vqe_ansatz_analysis(
                        matrix=matrix, ansatz=ansatz, optimizer=optimizer, seed=seed,
                        exact_ground_state=zeromode_classic, classical_expectation=classical_expectation, \
                        nmax=nmax, L=L)
                    
                    # Append the relative error and function call count
                    fidelities.append(fidelity_value)
                    function_calls.append(function_call_count)
                    all_relative_errors.append(error)
                
                # Calculate the average fidelity over the runs
                average_relative_error = np.mean(all_relative_errors)
                print(f"{pair_name} - Depth {current_depth}: Average relative error = {average_relative_error}")
        
                # Check if the average relative error meets the threshold
                if average_relative_error <= target_error_threshold:
                    min_depths[pair_name] = current_depth
                    converged = True
                    
                    # Identify the run with the lowest relative error at this depth
                    best_run_index = np.argmin(all_relative_errors)
                    best_error = all_relative_errors[best_run_index]
                    # best_zeromode = all_zeromodes[best_run_index]
                    best_function_calls = function_calls[best_run_index]
                    
                    # Print the run number and highest fidelity at optimal depth
                    print(f"Optimal depth {current_depth} for {pair_name} achieved lowest relative error = {best_error}")
                    print(f"Run number with lowest relative error: {best_run_index + 1}")
                    
                    # Calculate gate count and circuit depth for the ansatz at this depth
                    decomposed_ansatz = ansatz.decompose()  # Decompose to get actual gate operations
                    gate_count_dict = decomposed_ansatz.count_ops()
                    total_gates = sum(gate_count_dict.values())
                    circuit_depth = decomposed_ansatz.depth()
                    
                    # Store relative error and resource information
                    relative_errors[pair_name] = best_error
                    resource_info[pair_name] = {
                        'gate_count': total_gates,
                        'circuit_depth': circuit_depth,
                        'function_calls': best_function_calls
                    }

                    print(f"Relative error at optimal run for {pair_name}: {best_error}")
                    print(f"Resource estimates for {pair_name}: Gate count = {total_gates}, Circuit depth = {circuit_depth}, Function calls = {best_function_calls}")
                    
                    break  # Exit the loop if the threshold is met
        
                current_depth += 1  # Increase depth and try again
            
            # If the loop finishes and no convergence occurs, mark as "did not converge"
            if not converged:
                min_depths[pair_name] = "Did not converge"
                relative_errors[pair_name] = "Did not converge"
                resource_info[pair_name] = {
                    'gate_count': "N/A",
                    'circuit_depth': "N/A",
                    'function_calls': "N/A"
                }
                print(f"{pair_name} did not converge within {max_depth} depths.")
    
    return min_depths, relative_errors, resource_info

## Quantum (VQE) analysis

### For a = 1, b = 2 (N = 2)

### Get the matrix and the zeromode

In [18]:
## Get the matrix and zeromode
nmax = 3
a = 1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[  5.765625   -21.34578596]
 [-21.34578596  79.265625  ]]

The zeromode is:
[[-0.96558703]
 [-0.26008015]]

The value of <x^2> is:
0.41050077276583463



C:\Users\Owner\AppData\Local\Temp\ipykernel_32760\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Estimate resource requirements (fix at 5000 optimization iterations)

In [29]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_2, errors_N_2, resources_N_2 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_11088\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


SLSQP-RealAmplitudes - Depth 1: Average relative error = 0.11707369916486736


C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


SLSQP-RealAmplitudes - Depth 2: Average relative error = 7.841601337552767e-06
Optimal depth 2 for SLSQP-RealAmplitudes achieved lowest relative error = 5.336156839963159e-09
Run number with lowest relative error: 6
Relative error at optimal run for SLSQP-RealAmplitudes: 5.336156839963159e-09
Resource estimates for SLSQP-RealAmplitudes: Gate count = 3, Circuit depth = 3, Function calls = 27

Running VQE for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average relative error = 1.3693141044647793e-05
Optimal depth 1 for SLSQP-TwoLocal achieved lowest relative error = 5.272217918149814e-08
Run number with lowest relative error: 6
Relative error at optimal run for SLSQP-TwoLocal: 5.272217918149814e-08
Resource estimates for SLSQP-TwoLocal: Gate count = 2, Circuit depth = 2, Function calls = 20

Running VQE for optimizer-ansatz pair: SLSQP-EfficientSU2
SLSQP-EfficientSU2 - Depth 1: Average relative error = 0.11707369916486736


...

P_BFGS-EfficientSU2 - Depth 1: Average relative error = 0.1170676638171726


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 2: Average relative error = 0.6146617213952925


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 3: Average relative error = 7.420346770655438e-09
Optimal depth 3 for P_BFGS-EfficientSU2 achieved lowest relative error = 1.5244374687256997e-11
Run number with lowest relative error: 5
Relative error at optimal run for P_BFGS-EfficientSU2: 1.5244374687256997e-11
Resource estimates for P_BFGS-EfficientSU2: Gate count = 4, Circuit depth = 4, Function calls = 35


### Print resource estimates

In [32]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_2)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_2)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_2)
print()

The min. ansatz depth results are:
{'SLSQP-RealAmplitudes': 2, 'SLSQP-TwoLocal': 1, 'SLSQP-EfficientSU2': 2, 'P_BFGS-RealAmplitudes': 3, 'P_BFGS-TwoLocal': 1, 'P_BFGS-EfficientSU2': 3}

The relative errors are:
{'SLSQP-RealAmplitudes': 5.336156839963159e-09, 'SLSQP-TwoLocal': 5.272217918149814e-08, 'SLSQP-EfficientSU2': 5.336156839963159e-09, 'P_BFGS-RealAmplitudes': 1.5244374687256997e-11, 'P_BFGS-TwoLocal': 1.6332222168343596e-09, 'P_BFGS-EfficientSU2': 1.5244374687256997e-11}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 3, 'circuit_depth': 3, 'function_calls': 27}, 'SLSQP-TwoLocal': {'gate_count': 2, 'circuit_depth': 2, 'function_calls': 20}, 'SLSQP-EfficientSU2': {'gate_count': 3, 'circuit_depth': 3, 'function_calls': 27}, 'P_BFGS-RealAmplitudes': {'gate_count': 4, 'circuit_depth': 4, 'function_calls': 35}, 'P_BFGS-TwoLocal': {'gate_count': 2, 'circuit_depth': 2, 'function_calls': 27}, 'P_BFGS-EfficientSU2': {'gate_count': 4, 'circuit_depth': 4, 'function_ca

### Estimating resource requirements for POWELL and NELDER-MEAD (fix at 5000 optimization iterations)

In [21]:
## Get the optimizers
maxiter = 5000
optimizers = [NELDER_MEAD(maxiter = maxiter), POWELL(maxiter = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_2, errors_N_2, resources_N_2 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: NELDER_MEAD-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_32760\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


NELDER_MEAD-RealAmplitudes - Depth 1: Average relative error = 1.3933608688884431e-06
Optimal depth 1 for NELDER_MEAD-RealAmplitudes achieved lowest relative error = 6.243552580350284e-10
Run number with lowest relative error: 8
Relative error at optimal run for NELDER_MEAD-RealAmplitudes: 6.243552580350284e-10
Resource estimates for NELDER_MEAD-RealAmplitudes: Gate count = 2, Circuit depth = 2, Function calls = 107

Running VQE for optimizer-ansatz pair: NELDER_MEAD-TwoLocal
NELDER_MEAD-TwoLocal - Depth 1: Average relative error = 1.3933608688884431e-06
Optimal depth 1 for NELDER_MEAD-TwoLocal achieved lowest relative error = 6.243552580350284e-10
Run number with lowest relative error: 8
Relative error at optimal run for NELDER_MEAD-TwoLocal: 6.243552580350284e-10
Resource estimates for NELDER_MEAD-TwoLocal: Gate count = 2, Circuit depth = 2, Function calls = 107

Running VQE for optimizer-ansatz pair: NELDER_MEAD-EfficientSU2
NELDER_MEAD-EfficientSU2 - Depth 1: Average relative error

### Print resource estimates

In [24]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_2)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_2)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_2)
print()

The min. ansatz depth results are:
{'NELDER_MEAD-RealAmplitudes': 1, 'NELDER_MEAD-TwoLocal': 1, 'NELDER_MEAD-EfficientSU2': 1, 'POWELL-RealAmplitudes': 1, 'POWELL-TwoLocal': 1, 'POWELL-EfficientSU2': 1}

The relative errors are:
{'NELDER_MEAD-RealAmplitudes': 6.243552580350284e-10, 'NELDER_MEAD-TwoLocal': 6.243552580350284e-10, 'NELDER_MEAD-EfficientSU2': 6.243552580350284e-10, 'POWELL-RealAmplitudes': 6.284770039653191e-11, 'POWELL-TwoLocal': 5.191263271585534e-12, 'POWELL-EfficientSU2': 6.284770039653191e-11}

The resources required are:
{'NELDER_MEAD-RealAmplitudes': {'gate_count': 2, 'circuit_depth': 2, 'function_calls': 107}, 'NELDER_MEAD-TwoLocal': {'gate_count': 2, 'circuit_depth': 2, 'function_calls': 107}, 'NELDER_MEAD-EfficientSU2': {'gate_count': 2, 'circuit_depth': 2, 'function_calls': 107}, 'POWELL-RealAmplitudes': {'gate_count': 2, 'circuit_depth': 2, 'function_calls': 71}, 'POWELL-TwoLocal': {'gate_count': 2, 'circuit_depth': 2, 'function_calls': 38}, 'POWELL-EfficientSU

### For a = 1, b = 2 (N = 4)

### Get the matrix and the zeromode

In [26]:
## Get the matrix and zeromode
nmax = 7
a = 1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[   6.140625    -23.4671063    35.51760127   -9.22378041]
 [ -23.4671063    96.890625   -151.66269884  111.47028752]
 [  35.51760127 -151.66269884  376.140625   -354.47919269]
 [  -9.22378041  111.47028752 -354.47919269  972.890625  ]]

The zeromode is:
[[-0.97335411]
 [-0.22648723]
 [ 0.02489903]
 [ 0.02579447]]

The value of <x^2> is:
0.32868055910006333



C:\Users\Owner\AppData\Local\Temp\ipykernel_32760\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Estimate resource requirements (fix at 5000 optimization iterations)

In [38]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_4, errors_N_4, resources_N_4 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_11088\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


SLSQP-RealAmplitudes - Depth 1: Average relative error = 0.24968473488819098
SLSQP-RealAmplitudes - Depth 2: Average relative error = 0.3803728020238014
SLSQP-RealAmplitudes - Depth 3: Average relative error = 4.843210048540906e-05
Optimal depth 3 for SLSQP-RealAmplitudes achieved lowest relative error = 2.836542927502935e-06
Run number with lowest relative error: 3
Relative error at optimal run for SLSQP-RealAmplitudes: 2.836542927502935e-06
Resource estimates for SLSQP-RealAmplitudes: Gate count = 11, Circuit depth = 7, Function calls = 106

Running VQE for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average relative error = 7.51628079435751e-05
Optimal depth 1 for SLSQP-TwoLocal achieved lowest relative error = 8.632139444031718e-06
Run number with lowest relative error: 3
Relative error at optimal run for SLSQP-TwoLocal: 8.632139444031718e-06
Resource estimates for SLSQP-TwoLocal: Gate count = 5, Circuit depth = 3, Function calls = 80

Running VQE for optimizer-

C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:441: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)


SLSQP-EfficientSU2 - Depth 2: Average relative error = 0.03808474441722653


...

P_BFGS-EfficientSU2 - Depth 1: Average relative error = 0.2686750398148066


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 2: Average relative error = 0.13758628230959535


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 3: Average relative error = 2.58630848068677e-07
Optimal depth 3 for P_BFGS-EfficientSU2 achieved lowest relative error = 6.17281161420991e-10
Run number with lowest relative error: 1
Relative error at optimal run for P_BFGS-EfficientSU2: 6.17281161420991e-10
Resource estimates for P_BFGS-EfficientSU2: Gate count = 11, Circuit depth = 7, Function calls = 396


### Print resource estimates

In [42]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_4)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_4)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_4)
print()

The min. ansatz depth results are:
{'SLSQP-RealAmplitudes': 3, 'SLSQP-TwoLocal': 1, 'SLSQP-EfficientSU2': 3, 'P_BFGS-RealAmplitudes': 4, 'P_BFGS-TwoLocal': 1, 'P_BFGS-EfficientSU2': 3}

The relative errors are:
{'SLSQP-RealAmplitudes': 2.836542927502935e-06, 'SLSQP-TwoLocal': 8.632139444031718e-06, 'SLSQP-EfficientSU2': 7.130267372472693e-07, 'P_BFGS-RealAmplitudes': 7.442703715072253e-09, 'P_BFGS-TwoLocal': 5.607225521331557e-08, 'P_BFGS-EfficientSU2': 6.17281161420991e-10}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 11, 'circuit_depth': 7, 'function_calls': 106}, 'SLSQP-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 80}, 'SLSQP-EfficientSU2': {'gate_count': 11, 'circuit_depth': 7, 'function_calls': 142}, 'P_BFGS-RealAmplitudes': {'gate_count': 14, 'circuit_depth': 9, 'function_calls': 550}, 'P_BFGS-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 150}, 'P_BFGS-EfficientSU2': {'gate_count': 11, 'circuit_depth': 7, 'functio

### Estimating resource requirements for NELDER-MEAD and POWELL (fix at 5000 optimization iterations)

In [29]:
## Get the optimizers
maxiter = 5000
optimizers = [NELDER_MEAD(maxiter = maxiter), POWELL(maxiter = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_4, errors_N_4, resources_N_4 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: NELDER_MEAD-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_32760\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


NELDER_MEAD-RealAmplitudes - Depth 1: Average relative error = 6.923942654359994e-05
Optimal depth 1 for NELDER_MEAD-RealAmplitudes achieved lowest relative error = 1.916148148893346e-05
Run number with lowest relative error: 1
Relative error at optimal run for NELDER_MEAD-RealAmplitudes: 1.916148148893346e-05
Resource estimates for NELDER_MEAD-RealAmplitudes: Gate count = 5, Circuit depth = 3, Function calls = 175

Running VQE for optimizer-ansatz pair: NELDER_MEAD-TwoLocal
NELDER_MEAD-TwoLocal - Depth 1: Average relative error = 6.923942654359994e-05
Optimal depth 1 for NELDER_MEAD-TwoLocal achieved lowest relative error = 1.916148148893346e-05
Run number with lowest relative error: 1
Relative error at optimal run for NELDER_MEAD-TwoLocal: 1.916148148893346e-05
Resource estimates for NELDER_MEAD-TwoLocal: Gate count = 5, Circuit depth = 3, Function calls = 175

Running VQE for optimizer-ansatz pair: NELDER_MEAD-EfficientSU2
NELDER_MEAD-EfficientSU2 - Depth 1: Average relative error =

### Print resource estimates

In [32]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_4)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_4)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_4)
print()

The min. ansatz depth results are:
{'NELDER_MEAD-RealAmplitudes': 1, 'NELDER_MEAD-TwoLocal': 1, 'NELDER_MEAD-EfficientSU2': 1, 'POWELL-RealAmplitudes': 5, 'POWELL-TwoLocal': 1, 'POWELL-EfficientSU2': 3}

The relative errors are:
{'NELDER_MEAD-RealAmplitudes': 1.916148148893346e-05, 'NELDER_MEAD-TwoLocal': 1.916148148893346e-05, 'NELDER_MEAD-EfficientSU2': 1.916148148893346e-05, 'POWELL-RealAmplitudes': 4.658566225877019e-06, 'POWELL-TwoLocal': 5.363263817009758e-08, 'POWELL-EfficientSU2': 5.277416279603296e-06}

The resources required are:
{'NELDER_MEAD-RealAmplitudes': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 175}, 'NELDER_MEAD-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 175}, 'NELDER_MEAD-EfficientSU2': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 175}, 'POWELL-RealAmplitudes': {'gate_count': 17, 'circuit_depth': 11, 'function_calls': 702}, 'POWELL-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 438}, 'POWELL-Efficie

### For a = 1, b = 2 (N = 8)

### Get the matrix and the zeromode

In [34]:
## Get the matrix and zeromode
nmax = 15
a = 1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 6.14062500e+00 -2.34671063e+01  3.55176013e+01 -9.22378041e+00
  -3.13747510e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.34671063e+01  9.68906250e+01 -1.51662699e+02  1.11470288e+02
  -6.65558976e+01 -2.10468228e+01  0.00000000e+00  0.00000000e+00]
 [ 3.55176013e+01 -1.51662699e+02  4.02390625e+02 -3.54479193e+02
   1.63951212e+02 -2.30876387e+02 -6.98044143e+01  0.00000000e+00]
 [-9.22378041e+00  1.11470288e+02 -3.54479193e+02  1.05164062e+03
  -5.71772019e+02  1.15363556e+02 -5.86246269e+02 -1.71932490e+02]
 [-3.13747510e+00 -6.65558976e+01  1.63951212e+02 -5.71772019e+02
   2.23364062e+03 -7.19516989e+02 -1.36244266e+02 -1.24067405e+03]
 [ 0.00000000e+00 -2.10468228e+01 -2.30876387e+02  1.15363556e+02
  -7.19516989e+02  4.19739062e+03 -6.89706553e+02 -7.16860080e+02]
 [ 0.00000000e+00  0.00000000e+00 -6.98044143e+01 -5.86246269e+02
  -1.36244266e+02 -6.89706553e+02  6.56939062e+03 -1.15978184e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.7

C:\Users\Owner\AppData\Local\Temp\ipykernel_32760\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Estimate resource requirements (fix at 5000 optimization iterations)

In [48]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_8, errors_N_8, resources_N_8 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_11088\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


SLSQP-RealAmplitudes - Depth 1: Average relative error = 0.1769820399026217
SLSQP-RealAmplitudes - Depth 2: Average relative error = 0.032556950769379964
SLSQP-RealAmplitudes - Depth 3: Average relative error = 0.06432301042493446
SLSQP-RealAmplitudes - Depth 4: Average relative error = 8.04308259617955e-05
Optimal depth 4 for SLSQP-RealAmplitudes achieved lowest relative error = 1.7376325789516022e-05
Run number with lowest relative error: 9
Relative error at optimal run for SLSQP-RealAmplitudes: 1.7376325789516022e-05
Resource estimates for SLSQP-RealAmplitudes: Gate count = 27, Circuit depth = 17, Function calls = 569

Running VQE for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average relative error = 0.022191522924590562
SLSQP-TwoLocal - Depth 2: Average relative error = 0.01575752571302102
SLSQP-TwoLocal - Depth 3: Average relative error = 0.00011896187185419384
Optimal depth 3 for SLSQP-TwoLocal achieved lowest relative error = 2.464518617350669e-07
Run numbe

For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 4: Average relative error = 8.30570576411443e-05
Optimal depth 4 for SLSQP-EfficientSU2 achieved lowest relative error = 5.849134291772261e-07
Run number with lowest relative error: 2
Relative error at optimal run for SLSQP-EfficientSU2: 5.849134291772261e-07
Resource estimates for SLSQP-EfficientSU2: Gate count = 27, Circuit depth = 17, Function calls = 712

Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


...

P_BFGS-EfficientSU2 - Depth 4: Average relative error = 1.0612955650432512


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Average relative error = 0.21950880170222647


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Average relative error = 1.8017902984780919e-06
Optimal depth 6 for P_BFGS-EfficientSU2 achieved lowest relative error = 1.4131152509076865e-07
Run number with lowest relative error: 9
Relative error at optimal run for P_BFGS-EfficientSU2: 1.4131152509076865e-07
Resource estimates for P_BFGS-EfficientSU2: Gate count = 39, Circuit depth = 25, Function calls = 1870


### Print resource estimates

In [52]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_8)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_8)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_8)
print()

The min. ansatz depth results are:
{'SLSQP-RealAmplitudes': 4, 'SLSQP-TwoLocal': 3, 'SLSQP-EfficientSU2': 4, 'P_BFGS-RealAmplitudes': 4, 'P_BFGS-TwoLocal': 2, 'P_BFGS-EfficientSU2': 6}

The relative errors are:
{'SLSQP-RealAmplitudes': 1.7376325789516022e-05, 'SLSQP-TwoLocal': 2.464518617350669e-07, 'SLSQP-EfficientSU2': 5.849134291772261e-07, 'P_BFGS-RealAmplitudes': 1.2739785156561975e-07, 'P_BFGS-TwoLocal': 2.095845992341826e-05, 'P_BFGS-EfficientSU2': 1.4131152509076865e-07}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 27, 'circuit_depth': 17, 'function_calls': 569}, 'SLSQP-TwoLocal': {'gate_count': 21, 'circuit_depth': 13, 'function_calls': 501}, 'SLSQP-EfficientSU2': {'gate_count': 27, 'circuit_depth': 17, 'function_calls': 712}, 'P_BFGS-RealAmplitudes': {'gate_count': 27, 'circuit_depth': 17, 'function_calls': 1264}, 'P_BFGS-TwoLocal': {'gate_count': 15, 'circuit_depth': 9, 'function_calls': 2750}, 'P_BFGS-EfficientSU2': {'gate_count': 39, 'circuit_depth'

### Estimating resource requirements for NELDER-MEAD and POWELL (fix at 5000 optimization iterations)

In [37]:
## Get the optimizers
maxiter = 5000
optimizers = [NELDER_MEAD(maxiter = maxiter), POWELL(maxiter = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_8, errors_N_8, resources_N_8 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: NELDER_MEAD-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_32760\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


NELDER_MEAD-RealAmplitudes - Depth 1: Average relative error = 0.014390858346036397
NELDER_MEAD-RealAmplitudes - Depth 2: Average relative error = 1.0003297803320783
NELDER_MEAD-RealAmplitudes - Depth 3: Average relative error = 3.7053969282804737
NELDER_MEAD-RealAmplitudes - Depth 4: Average relative error = 0.333632209676466
NELDER_MEAD-RealAmplitudes - Depth 5: Average relative error = 1.670410489028559
NELDER_MEAD-RealAmplitudes - Depth 6: Average relative error = 3.752516081291339
NELDER_MEAD-RealAmplitudes - Depth 7: Average relative error = 1.0408802403429622
NELDER_MEAD-RealAmplitudes - Depth 8: Average relative error = 2.6612221393748206
NELDER_MEAD-RealAmplitudes - Depth 9: Average relative error = 1.5728412312918139
NELDER_MEAD-RealAmplitudes - Depth 10: Average relative error = 3.2346494031593567
NELDER_MEAD-RealAmplitudes did not converge within 10 depths.

Running VQE for optimizer-ansatz pair: NELDER_MEAD-TwoLocal
NELDER_MEAD-TwoLocal - Depth 1: Average relative error = 

### Print resource estimates

In [40]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_8)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_8)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_8)
print()

The min. ansatz depth results are:
{'NELDER_MEAD-RealAmplitudes': 'Did not converge', 'NELDER_MEAD-TwoLocal': 'Did not converge', 'NELDER_MEAD-EfficientSU2': 'Did not converge', 'POWELL-RealAmplitudes': 'Did not converge', 'POWELL-TwoLocal': 'Did not converge', 'POWELL-EfficientSU2': 'Did not converge'}

The relative errors are:
{'NELDER_MEAD-RealAmplitudes': 'Did not converge', 'NELDER_MEAD-TwoLocal': 'Did not converge', 'NELDER_MEAD-EfficientSU2': 'Did not converge', 'POWELL-RealAmplitudes': 'Did not converge', 'POWELL-TwoLocal': 'Did not converge', 'POWELL-EfficientSU2': 'Did not converge'}

The resources required are:
{'NELDER_MEAD-RealAmplitudes': {'gate_count': 'N/A', 'circuit_depth': 'N/A', 'function_calls': 'N/A'}, 'NELDER_MEAD-TwoLocal': {'gate_count': 'N/A', 'circuit_depth': 'N/A', 'function_calls': 'N/A'}, 'NELDER_MEAD-EfficientSU2': {'gate_count': 'N/A', 'circuit_depth': 'N/A', 'function_calls': 'N/A'}, 'POWELL-RealAmplitudes': {'gate_count': 'N/A', 'circuit_depth': 'N/A', 

### For a = 1, b = 2 (N = 16)

### Get the matrix and the zeromode

In [19]:
## Get the matrix and zeromode
nmax = 31
a = 1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 6.14062500e+00 -2.34671063e+01  3.55176013e+01 -9.22378041e+00
  -3.13747510e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.34671063e+01  9.68906250e+01 -1.51662699e+02  1.11470288e+02
  -6.65558976e+01 -2.10468228e+01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.55176013e+01 -1.51662699e+02  4.02390625e+02 -3.54479193e+02
   1.63951212e+02 -2.30876387e+02 -6.98044143e+01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-9.22378041e+00  1.11470288e+02 -3.54479193e+02  1.05164062e+03
  -5.71772019e+02  1.15363556e+02 -5.86246269e+02 -1.71932490e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000

C:\Users\Owner\AppData\Local\Temp\ipykernel_47076\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The value of <x^2> is:
0.3662333595845026



### Estimate resource requirements (fix at 5000 optimization iterations)

In [22]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_16, errors_N_16, resources_N_16 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_47076\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]
C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


SLSQP-RealAmplitudes - Depth 1: Average relative error = 0.19806243802172335
SLSQP-RealAmplitudes - Depth 2: Average relative error = 0.026695306176365858
SLSQP-RealAmplitudes - Depth 3: Average relative error = 0.06802217243489109
SLSQP-RealAmplitudes - Depth 4: Average relative error = 0.009148945466770012
Optimal depth 4 for SLSQP-RealAmplitudes achieved lowest relative error = 0.002754492118953014
Run number with lowest relative error: 1
Relative error at optimal run for SLSQP-RealAmplitudes: 0.002754492118953014
Resource estimates for SLSQP-RealAmplitudes: Gate count = 44, Circuit depth = 22, Function calls = 1438

Running VQE for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average relative error = 0.020332486217414247
SLSQP-TwoLocal - Depth 2: Average relative error = 0.016364046315085336
SLSQP-TwoLocal - Depth 3: Average relative error = 0.008488501626843576
Optimal depth 3 for SLSQP-TwoLocal achieved lowest relative error = 0.0007149109040982413
Run number w

C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:441: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)


SLSQP-EfficientSU2 - Depth 1: Average relative error = 3.022350801962421
SLSQP-EfficientSU2 - Depth 2: Average relative error = 0.2697766119558854
SLSQP-EfficientSU2 - Depth 3: Average relative error = 0.021852279475247937
SLSQP-EfficientSU2 - Depth 4: Average relative error = 0.11994175173512234


...

P_BFGS-EfficientSU2 - Depth 8: Average relative error = 1.754086033668619


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 9: Average relative error = 0.6713909103128061


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 10: Average relative error = 1.9636128213854653
P_BFGS-EfficientSU2 did not converge within 10 depths.


### Print the results

In [25]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_16)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_16)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_16)
print()

The min. ansatz depth results are:
{'SLSQP-RealAmplitudes': 4, 'SLSQP-TwoLocal': 3, 'SLSQP-EfficientSU2': 5, 'P_BFGS-RealAmplitudes': 'Did not converge', 'P_BFGS-TwoLocal': 3, 'P_BFGS-EfficientSU2': 'Did not converge'}

The relative errors are:
{'SLSQP-RealAmplitudes': 0.002754492118953014, 'SLSQP-TwoLocal': 0.0007149109040982413, 'SLSQP-EfficientSU2': 1.7298005847677678e-05, 'P_BFGS-RealAmplitudes': 'Did not converge', 'P_BFGS-TwoLocal': 0.0021107143976370236, 'P_BFGS-EfficientSU2': 'Did not converge'}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 44, 'circuit_depth': 22, 'function_calls': 1438}, 'SLSQP-TwoLocal': {'gate_count': 34, 'circuit_depth': 17, 'function_calls': 1686}, 'SLSQP-EfficientSU2': {'gate_count': 44, 'circuit_depth': 26, 'function_calls': 2647}, 'P_BFGS-RealAmplitudes': {'gate_count': 'N/A', 'circuit_depth': 'N/A', 'function_calls': 'N/A'}, 'P_BFGS-TwoLocal': {'gate_count': 34, 'circuit_depth': 17, 'function_calls': 5032}, 'P_BFGS-EfficientSU2'

### For a = -1, b = 2 (N = 2)

### Get the matrix and the zeromode

In [55]:
## Get the matrix and zeromode
nmax = 3
a = -1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 17.015625   -37.25568853]
 [-37.25568853  84.515625  ]]

The zeromode is:
[[-0.91416021]
 [-0.40535307]]

The value of <x^2> is:
0.48931145833928297



C:\Users\Owner\AppData\Local\Temp\ipykernel_11088\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Estimate resource requirements (fix at 5000 optimization iterations)

In [58]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_2, errors_N_2, resources_N_2 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_11088\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


SLSQP-RealAmplitudes - Depth 1: Average relative error = 0.14653160143751628
SLSQP-RealAmplitudes - Depth 2: Average relative error = 1.8879777725189022e-05
Optimal depth 2 for SLSQP-RealAmplitudes achieved lowest relative error = 4.292026354756172e-09
Run number with lowest relative error: 6
Relative error at optimal run for SLSQP-RealAmplitudes: 4.292026354756172e-09
Resource estimates for SLSQP-RealAmplitudes: Gate count = 3, Circuit depth = 3, Function calls = 23

Running VQE for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average relative error = 1.3148079243836353e-05
Optimal depth 1 for SLSQP-TwoLocal achieved lowest relative error = 6.894609420541554e-10
Run number with lowest relative error: 8
Relative error at optimal run for SLSQP-TwoLocal: 6.894609420541554e-10
Resource estimates for SLSQP-TwoLocal: Gate count = 2, Circuit depth = 2, Function calls = 21

Running VQE for optimizer-ansatz pair: SLSQP-EfficientSU2
SLSQP-EfficientSU2 - Depth 1: Average relat

For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 2: Average relative error = 1.8879777725189022e-05
Optimal depth 2 for SLSQP-EfficientSU2 achieved lowest relative error = 4.292026354756172e-09
Run number with lowest relative error: 6
Relative error at optimal run for SLSQP-EfficientSU2: 4.292026354756172e-09
Resource estimates for SLSQP-EfficientSU2: Gate count = 3, Circuit depth = 3, Function calls = 23

Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


...

P_BFGS-EfficientSU2 - Depth 1: Average relative error = 0.14653157057056895


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 2: Average relative error = 0.46734221695396005


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 3: Average relative error = 6.7762359389037945e-09
Optimal depth 3 for P_BFGS-EfficientSU2 achieved lowest relative error = 5.962260603191657e-10
Run number with lowest relative error: 4
Relative error at optimal run for P_BFGS-EfficientSU2: 5.962260603191657e-10
Resource estimates for P_BFGS-EfficientSU2: Gate count = 4, Circuit depth = 4, Function calls = 35


### Print resource estimates

In [61]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_2)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_2)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_2)
print()

The min. ansatz depth results are:
{'SLSQP-RealAmplitudes': 2, 'SLSQP-TwoLocal': 1, 'SLSQP-EfficientSU2': 2, 'P_BFGS-RealAmplitudes': 3, 'P_BFGS-TwoLocal': 1, 'P_BFGS-EfficientSU2': 3}

The relative errors are:
{'SLSQP-RealAmplitudes': 4.292026354756172e-09, 'SLSQP-TwoLocal': 6.894609420541554e-10, 'SLSQP-EfficientSU2': 4.292026354756172e-09, 'P_BFGS-RealAmplitudes': 5.962260603191657e-10, 'P_BFGS-TwoLocal': 5.070049503793998e-10, 'P_BFGS-EfficientSU2': 5.962260603191657e-10}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 3, 'circuit_depth': 3, 'function_calls': 23}, 'SLSQP-TwoLocal': {'gate_count': 2, 'circuit_depth': 2, 'function_calls': 21}, 'SLSQP-EfficientSU2': {'gate_count': 3, 'circuit_depth': 3, 'function_calls': 23}, 'P_BFGS-RealAmplitudes': {'gate_count': 4, 'circuit_depth': 4, 'function_calls': 35}, 'P_BFGS-TwoLocal': {'gate_count': 2, 'circuit_depth': 2, 'function_calls': 18}, 'P_BFGS-EfficientSU2': {'gate_count': 4, 'circuit_depth': 4, 'function_calls

### For a = -1, b = 2 (N = 4)

### Get the matrix and the zeromode

In [64]:
## Get the matrix and zeromode
nmax = 7
a = -1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[  17.390625    -41.49832922   41.64132563   -2.51557647]
 [ -41.49832922  138.140625   -204.49024847  154.16103593]
 [  41.64132563 -204.49024847  387.390625   -424.31381877]
 [  -2.51557647  154.16103593 -424.31381877  790.140625  ]]

The zeromode is:
[[-0.80492107]
 [-0.55178885]
 [-0.21791911]
 [-0.01193297]]

The value of <x^2> is:
0.7580435055926756



C:\Users\Owner\AppData\Local\Temp\ipykernel_11088\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Estimate resource requirements (fix at 5000 optimization iterations)

In [67]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_4, errors_N_4, resources_N_4 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_11088\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


SLSQP-RealAmplitudes - Depth 1: Average relative error = 0.026685304352362754
SLSQP-RealAmplitudes - Depth 2: Average relative error = 0.11421118747026257
SLSQP-RealAmplitudes - Depth 3: Average relative error = 0.04406038038468647
SLSQP-RealAmplitudes - Depth 4: Average relative error = 3.30834530961778e-05
Optimal depth 4 for SLSQP-RealAmplitudes achieved lowest relative error = 8.867874604269487e-07
Run number with lowest relative error: 6
Relative error at optimal run for SLSQP-RealAmplitudes: 8.867874604269487e-07
Resource estimates for SLSQP-RealAmplitudes: Gate count = 14, Circuit depth = 9, Function calls = 236

Running VQE for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average relative error = 9.713419418917283e-05
Optimal depth 1 for SLSQP-TwoLocal achieved lowest relative error = 3.8593836728381256e-07
Run number with lowest relative error: 7
Relative error at optimal run for SLSQP-TwoLocal: 3.8593836728381256e-07
Resource estimates for SLSQP-TwoLocal: G

For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 3: Average relative error = 4.2503207821117936e-05
Optimal depth 3 for SLSQP-EfficientSU2 achieved lowest relative error = 3.0729606825647177e-06
Run number with lowest relative error: 3
Relative error at optimal run for SLSQP-EfficientSU2: 3.0729606825647177e-06
Resource estimates for SLSQP-EfficientSU2: Gate count = 11, Circuit depth = 7, Function calls = 213

Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


...

P_BFGS-EfficientSU2 - Depth 1: Average relative error = 0.02664886345392259


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 2: Average relative error = 0.2539276784277459


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 3: Average relative error = 1.644875816634471e-07
Optimal depth 3 for P_BFGS-EfficientSU2 achieved lowest relative error = 1.2894078399654205e-08
Run number with lowest relative error: 2
Relative error at optimal run for P_BFGS-EfficientSU2: 1.2894078399654205e-08
Resource estimates for P_BFGS-EfficientSU2: Gate count = 11, Circuit depth = 7, Function calls = 207


### Print resource estimates

In [70]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_4)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_4)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_4)
print()

The min. ansatz depth results are:
{'SLSQP-RealAmplitudes': 4, 'SLSQP-TwoLocal': 1, 'SLSQP-EfficientSU2': 3, 'P_BFGS-RealAmplitudes': 4, 'P_BFGS-TwoLocal': 1, 'P_BFGS-EfficientSU2': 3}

The relative errors are:
{'SLSQP-RealAmplitudes': 8.867874604269487e-07, 'SLSQP-TwoLocal': 3.8593836728381256e-07, 'SLSQP-EfficientSU2': 3.0729606825647177e-06, 'P_BFGS-RealAmplitudes': 2.243422859439806e-08, 'P_BFGS-TwoLocal': 5.741521609805066e-09, 'P_BFGS-EfficientSU2': 1.2894078399654205e-08}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 14, 'circuit_depth': 9, 'function_calls': 236}, 'SLSQP-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 109}, 'SLSQP-EfficientSU2': {'gate_count': 11, 'circuit_depth': 7, 'function_calls': 213}, 'P_BFGS-RealAmplitudes': {'gate_count': 14, 'circuit_depth': 9, 'function_calls': 473}, 'P_BFGS-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 130}, 'P_BFGS-EfficientSU2': {'gate_count': 11, 'circuit_depth': 7, 'fu

### For a = -1, b = 2 (N = 8)

### Get the matrix and the zeromode

In [73]:
## Get the matrix and zeromode
nmax = 15
a = -1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 1.73906250e+01 -4.14983292e+01  4.16413256e+01 -2.51557647e+00
  -3.13747510e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-4.14983292e+01  1.38140625e+02 -2.04490248e+02  1.54161036e+02
  -3.10594189e+01 -2.10468228e+01  0.00000000e+00  0.00000000e+00]
 [ 4.16413256e+01 -2.04490248e+02  4.13640625e+02 -4.62654398e+02
   2.97161572e+02 -1.33665277e+02 -6.98044143e+01  0.00000000e+00]
 [-2.51557647e+00  1.54161036e+02 -4.62654398e+02  9.24890625e+02
  -7.68209032e+02  4.17083625e+02 -3.82334523e+02 -1.71932490e+02]
 [-3.13747510e+00 -3.10594189e+01  2.97161572e+02 -7.68209032e+02
   1.81289062e+03 -1.04918443e+03  4.35981651e+02 -8.73066927e+02]
 [ 0.00000000e+00 -2.10468228e+01 -1.33665277e+02  4.17083625e+02
  -1.04918443e+03  3.27864062e+03 -1.20958947e+03  2.51869758e+02]
 [ 0.00000000e+00  0.00000000e+00 -6.98044143e+01 -3.82334523e+02
   4.35981651e+02 -1.20958947e+03  4.90064062e+03 -1.53414981e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.7

C:\Users\Owner\AppData\Local\Temp\ipykernel_11088\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Estimate resource requirements (fix at 5000 optimization iterations)

In [76]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_8, errors_N_8, resources_N_8 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_11088\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


SLSQP-RealAmplitudes - Depth 1: Average relative error = 0.3942387861866542
SLSQP-RealAmplitudes - Depth 2: Average relative error = 0.005235440870832742
Optimal depth 2 for SLSQP-RealAmplitudes achieved lowest relative error = 2.404972472478609e-06
Run number with lowest relative error: 6
Relative error at optimal run for SLSQP-RealAmplitudes: 2.404972472478609e-06
Resource estimates for SLSQP-RealAmplitudes: Gate count = 15, Circuit depth = 9, Function calls = 422

Running VQE for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average relative error = 0.05332697997180615
SLSQP-TwoLocal - Depth 2: Average relative error = 0.0028377553983254906
Optimal depth 2 for SLSQP-TwoLocal achieved lowest relative error = 8.084603690787865e-05
Run number with lowest relative error: 1
Relative error at optimal run for SLSQP-TwoLocal: 8.084603690787865e-05
Resource estimates for SLSQP-TwoLocal: Gate count = 15, Circuit depth = 9, Function calls = 643

Running VQE for optimizer-ansa

For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 4: Average relative error = 5.944625890666698e-05
Optimal depth 4 for SLSQP-EfficientSU2 achieved lowest relative error = 1.4669562483966657e-06
Run number with lowest relative error: 2
Relative error at optimal run for SLSQP-EfficientSU2: 1.4669562483966657e-06
Resource estimates for SLSQP-EfficientSU2: Gate count = 27, Circuit depth = 17, Function calls = 793

Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


...

P_BFGS-EfficientSU2 - Depth 4: Average relative error = 0.24789953757558117


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Average relative error = 0.6047195277411983


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Average relative error = 8.835470724090545e-07
Optimal depth 6 for P_BFGS-EfficientSU2 achieved lowest relative error = 1.410191432154513e-07
Run number with lowest relative error: 3
Relative error at optimal run for P_BFGS-EfficientSU2: 1.410191432154513e-07
Resource estimates for P_BFGS-EfficientSU2: Gate count = 39, Circuit depth = 25, Function calls = 2684


### Print resource estimates

In [79]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_8)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_8)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_8)
print()

The min. ansatz depth results are:
{'SLSQP-RealAmplitudes': 2, 'SLSQP-TwoLocal': 2, 'SLSQP-EfficientSU2': 4, 'P_BFGS-RealAmplitudes': 4, 'P_BFGS-TwoLocal': 2, 'P_BFGS-EfficientSU2': 6}

The relative errors are:
{'SLSQP-RealAmplitudes': 2.404972472478609e-06, 'SLSQP-TwoLocal': 8.084603690787865e-05, 'SLSQP-EfficientSU2': 1.4669562483966657e-06, 'P_BFGS-RealAmplitudes': 2.8526900864602765e-07, 'P_BFGS-TwoLocal': 2.411690472012576e-06, 'P_BFGS-EfficientSU2': 1.410191432154513e-07}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 15, 'circuit_depth': 9, 'function_calls': 422}, 'SLSQP-TwoLocal': {'gate_count': 15, 'circuit_depth': 9, 'function_calls': 643}, 'SLSQP-EfficientSU2': {'gate_count': 27, 'circuit_depth': 17, 'function_calls': 793}, 'P_BFGS-RealAmplitudes': {'gate_count': 27, 'circuit_depth': 17, 'function_calls': 2864}, 'P_BFGS-TwoLocal': {'gate_count': 15, 'circuit_depth': 9, 'function_calls': 1200}, 'P_BFGS-EfficientSU2': {'gate_count': 39, 'circuit_depth': 2

### For a = -1, b = 2 (N = 16)

### Get the matrix and the zeromode

In [28]:
## Get the matrix and zeromode
nmax = 31
a = -1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 1.73906250e+01 -4.14983292e+01  4.16413256e+01 -2.51557647e+00
  -3.13747510e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-4.14983292e+01  1.38140625e+02 -2.04490248e+02  1.54161036e+02
  -3.10594189e+01 -2.10468228e+01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 4.16413256e+01 -2.04490248e+02  4.13640625e+02 -4.62654398e+02
   2.97161572e+02 -1.33665277e+02 -6.98044143e+01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.51557647e+00  1.54161036e+02 -4.62654398e+02  9.24890625e+02
  -7.68209032e+02  4.17083625e+02 -3.82334523e+02 -1.71932490e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000

C:\Users\Owner\AppData\Local\Temp\ipykernel_47076\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The value of <x^2> is:
0.6471064574160962



### Estimating resource requirements (fix at 5000 optimization iterations)

In [31]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_16, errors_N_16, resources_N_16 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_47076\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


SLSQP-RealAmplitudes - Depth 1: Average relative error = 0.11682480885921372
SLSQP-RealAmplitudes - Depth 2: Average relative error = 0.0516398448802874
SLSQP-RealAmplitudes - Depth 3: Average relative error = 0.24773510679199048
SLSQP-RealAmplitudes - Depth 4: Average relative error = 0.0549747329322243
SLSQP-RealAmplitudes - Depth 5: Average relative error = 0.002526710905965401
Optimal depth 5 for SLSQP-RealAmplitudes achieved lowest relative error = 1.2841541244993534e-05
Run number with lowest relative error: 6
Relative error at optimal run for SLSQP-RealAmplitudes: 1.2841541244993534e-05
Resource estimates for SLSQP-RealAmplitudes: Gate count = 54, Circuit depth = 27, Function calls = 1248

Running VQE for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average relative error = 0.09056961375837055
SLSQP-TwoLocal - Depth 2: Average relative error = 0.022893532856594692
SLSQP-TwoLocal - Depth 3: Average relative error = 0.007017152441387608
Optimal depth 3 for SLSQP

For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 4: Average relative error = 0.0016695536998567722
Optimal depth 4 for SLSQP-EfficientSU2 achieved lowest relative error = 6.414381781907047e-05
Run number with lowest relative error: 10
Relative error at optimal run for SLSQP-EfficientSU2: 6.414381781907047e-05
Resource estimates for SLSQP-EfficientSU2: Gate count = 36, Circuit depth = 21, Function calls = 1919

Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


...

P_BFGS-EfficientSU2 - Depth 8: Average relative error = 1.336612660284348


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 9: Average relative error = 4.584898674644727


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 10: Average relative error = 0.7018550415775751
P_BFGS-EfficientSU2 did not converge within 10 depths.


### Print the results

In [34]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_16)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_16)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_16)
print()

The min. ansatz depth results are:
{'SLSQP-RealAmplitudes': 5, 'SLSQP-TwoLocal': 3, 'SLSQP-EfficientSU2': 4, 'P_BFGS-RealAmplitudes': 'Did not converge', 'P_BFGS-TwoLocal': 2, 'P_BFGS-EfficientSU2': 'Did not converge'}

The relative errors are:
{'SLSQP-RealAmplitudes': 1.2841541244993534e-05, 'SLSQP-TwoLocal': 0.0024836094545450543, 'SLSQP-EfficientSU2': 6.414381781907047e-05, 'P_BFGS-RealAmplitudes': 'Did not converge', 'P_BFGS-TwoLocal': 0.00012289495854951593, 'P_BFGS-EfficientSU2': 'Did not converge'}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 54, 'circuit_depth': 27, 'function_calls': 1248}, 'SLSQP-TwoLocal': {'gate_count': 34, 'circuit_depth': 17, 'function_calls': 1317}, 'SLSQP-EfficientSU2': {'gate_count': 36, 'circuit_depth': 21, 'function_calls': 1919}, 'P_BFGS-RealAmplitudes': {'gate_count': 'N/A', 'circuit_depth': 'N/A', 'function_calls': 'N/A'}, 'P_BFGS-TwoLocal': {'gate_count': 24, 'circuit_depth': 12, 'function_calls': 4121}, 'P_BFGS-EfficientSU

### For a = 1, b = 0 (N = 2)

### Get the matrix and the zeromode

In [19]:
## Get the matrix and zeromode
nmax = 3
a = 1
gamma = 1
dx = 0.01
c = 0
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[  6.75       -25.45584412]
 [-25.45584412 102.75      ]]

The zeromode is:
[[-0.97042573]
 [-0.24139987]]

The value of <x^2> is:
0.40008586593679



C:\Users\Owner\AppData\Local\Temp\ipykernel_26868\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Estimate resource requirements (fix at 5000 optimization iterations)

In [22]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_2, errors_N_2, resources_N_2 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_26868\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


SLSQP-RealAmplitudes - Depth 1: Average relative error = 0.07488136401807644
SLSQP-RealAmplitudes - Depth 2: Average relative error = 7.670365683273846e-06
Optimal depth 2 for SLSQP-RealAmplitudes achieved lowest relative error = 5.9895000490341794e-09
Run number with lowest relative error: 6
Relative error at optimal run for SLSQP-RealAmplitudes: 5.9895000490341794e-09
Resource estimates for SLSQP-RealAmplitudes: Gate count = 3, Circuit depth = 3, Function calls = 27

Running VQE for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average relative error = 2.1927352946061566e-05
Optimal depth 1 for SLSQP-TwoLocal achieved lowest relative error = 7.651001395424214e-09
Run number with lowest relative error: 7
Relative error at optimal run for SLSQP-TwoLocal: 7.651001395424214e-09
Resource estimates for SLSQP-TwoLocal: Gate count = 2, Circuit depth = 2, Function calls = 21

Running VQE for optimizer-ansatz pair: SLSQP-EfficientSU2
SLSQP-EfficientSU2 - Depth 1: Average rela

For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 2: Average relative error = 7.670365683273846e-06
Optimal depth 2 for SLSQP-EfficientSU2 achieved lowest relative error = 5.9895000490341794e-09
Run number with lowest relative error: 6
Relative error at optimal run for SLSQP-EfficientSU2: 5.9895000490341794e-09
Resource estimates for SLSQP-EfficientSU2: Gate count = 3, Circuit depth = 3, Function calls = 27

Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


...

P_BFGS-EfficientSU2 - Depth 1: Average relative error = 0.11230562979079899


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 2: Average relative error = 0.6384718961948734


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 3: Average relative error = 5.202052170277253e-09
Optimal depth 3 for P_BFGS-EfficientSU2 achieved lowest relative error = 4.158149946195491e-10
Run number with lowest relative error: 2
Relative error at optimal run for P_BFGS-EfficientSU2: 4.158149946195491e-10
Resource estimates for P_BFGS-EfficientSU2: Gate count = 4, Circuit depth = 4, Function calls = 35


### Print resource estimates

In [25]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_2)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_2)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_2)
print()

The min. ansatz depth results are:
{'SLSQP-RealAmplitudes': 2, 'SLSQP-TwoLocal': 1, 'SLSQP-EfficientSU2': 2, 'P_BFGS-RealAmplitudes': 3, 'P_BFGS-TwoLocal': 1, 'P_BFGS-EfficientSU2': 3}

The relative errors are:
{'SLSQP-RealAmplitudes': 5.9895000490341794e-09, 'SLSQP-TwoLocal': 7.651001395424214e-09, 'SLSQP-EfficientSU2': 5.9895000490341794e-09, 'P_BFGS-RealAmplitudes': 4.158149946195491e-10, 'P_BFGS-TwoLocal': 8.928951812668842e-10, 'P_BFGS-EfficientSU2': 4.158149946195491e-10}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 3, 'circuit_depth': 3, 'function_calls': 27}, 'SLSQP-TwoLocal': {'gate_count': 2, 'circuit_depth': 2, 'function_calls': 21}, 'SLSQP-EfficientSU2': {'gate_count': 3, 'circuit_depth': 3, 'function_calls': 27}, 'P_BFGS-RealAmplitudes': {'gate_count': 4, 'circuit_depth': 4, 'function_calls': 35}, 'P_BFGS-TwoLocal': {'gate_count': 2, 'circuit_depth': 2, 'function_calls': 18}, 'P_BFGS-EfficientSU2': {'gate_count': 4, 'circuit_depth': 4, 'function_cal

### For a = 1, b = 0 (N = 4)

### Get the matrix and the zeromode

In [29]:
## Get the matrix and zeromode
nmax = 7
a = 1
gamma = 1
dx = 0.01
c = 0
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[   6.75        -25.45584412   18.37117307    0.        ]
 [ -25.45584412  129.75       -173.20508076   71.15124735]
 [  18.37117307 -173.20508076  448.75       -449.13249715]
 [   0.           71.15124735 -449.13249715  837.75      ]]

The zeromode is:
[[-0.92504521]
 [-0.34807967]
 [-0.14437197]
 [-0.04783967]]

The value of <x^2> is:
0.6419765393615697



C:\Users\Owner\AppData\Local\Temp\ipykernel_26868\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Estimate resource requirements (fix at 5000 optimization iterations)

In [32]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_4, errors_N_4, resources_N_4 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_26868\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]
C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


SLSQP-RealAmplitudes - Depth 1: Average relative error = 0.3792741770135687
SLSQP-RealAmplitudes - Depth 2: Average relative error = 0.1513595301086548
SLSQP-RealAmplitudes - Depth 3: Average relative error = 3.274933414107105e-05
Optimal depth 3 for SLSQP-RealAmplitudes achieved lowest relative error = 2.8237250946032572e-06
Run number with lowest relative error: 1
Relative error at optimal run for SLSQP-RealAmplitudes: 2.8237250946032572e-06
Resource estimates for SLSQP-RealAmplitudes: Gate count = 11, Circuit depth = 7, Function calls = 142

Running VQE for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average relative error = 5.487690955940053e-05
Optimal depth 1 for SLSQP-TwoLocal achieved lowest relative error = 3.033551887178738e-06
Run number with lowest relative error: 5
Relative error at optimal run for SLSQP-TwoLocal: 3.033551887178738e-06
Resource estimates for SLSQP-TwoLocal: Gate count = 5, Circuit depth = 3, Function calls = 159

Running VQE for optimiz

C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:441: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)


SLSQP-EfficientSU2 - Depth 2: Average relative error = 0.06104622680705961
SLSQP-EfficientSU2 - Depth 3: Average relative error = 0.04572620774567097


...

P_BFGS-EfficientSU2 - Depth 1: Average relative error = 0.16323062953907255


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 2: Average relative error = 0.3122692710454967


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 3: Average relative error = 1.6241554383556552e-07
Optimal depth 3 for P_BFGS-EfficientSU2 achieved lowest relative error = 3.520059432603429e-09
Run number with lowest relative error: 1
Relative error at optimal run for P_BFGS-EfficientSU2: 3.520059432603429e-09
Resource estimates for P_BFGS-EfficientSU2: Gate count = 11, Circuit depth = 7, Function calls = 243


### Print resource estimates

In [35]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_4)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_4)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_4)
print()

The min. ansatz depth results are:
{'SLSQP-RealAmplitudes': 3, 'SLSQP-TwoLocal': 1, 'SLSQP-EfficientSU2': 4, 'P_BFGS-RealAmplitudes': 4, 'P_BFGS-TwoLocal': 1, 'P_BFGS-EfficientSU2': 3}

The relative errors are:
{'SLSQP-RealAmplitudes': 2.8237250946032572e-06, 'SLSQP-TwoLocal': 3.033551887178738e-06, 'SLSQP-EfficientSU2': 1.7148993657021465e-06, 'P_BFGS-RealAmplitudes': 2.5750902542678553e-08, 'P_BFGS-TwoLocal': 2.269803009897159e-09, 'P_BFGS-EfficientSU2': 3.520059432603429e-09}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 11, 'circuit_depth': 7, 'function_calls': 142}, 'SLSQP-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 159}, 'SLSQP-EfficientSU2': {'gate_count': 14, 'circuit_depth': 9, 'function_calls': 135}, 'P_BFGS-RealAmplitudes': {'gate_count': 14, 'circuit_depth': 9, 'function_calls': 396}, 'P_BFGS-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 160}, 'P_BFGS-EfficientSU2': {'gate_count': 11, 'circuit_depth': 7, 'fu

### For a = 1, b = 0 (N = 8)

### Get the matrix and the zeromode

In [38]:
## Get the matrix and zeromode
nmax = 15
a = 1
gamma = 1
dx = 0.01
c = 0
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[    6.75         -25.45584412    18.37117307     0.
      0.             0.             0.             0.        ]
 [  -25.45584412   129.75        -173.20508076    71.15124735
      0.             0.             0.             0.        ]
 [   18.37117307  -173.20508076   448.75        -449.13249715
    153.70426149     0.             0.             0.        ]
 [    0.            71.15124735  -449.13249715   963.75
   -853.09788418   266.22359024     0.             0.        ]
 [    0.             0.           153.70426149  -853.09788418
   1674.75       -1385.07761515   408.7327978      0.        ]
 [    0.             0.             0.           266.22359024
  -1385.07761515  2581.75       -2045.06430217   581.2379031 ]
 [    0.             0.             0.             0.
    408.7327978  -2045.06430217  3684.75       -2833.05488828]
 [    0.             0.             0.             0.
      0.           581.2379031  -2833.05488828  4443.75      ]]

The zeromode 

C:\Users\Owner\AppData\Local\Temp\ipykernel_26868\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Estimate resource requirements (fix at 5000 optimization iterations)

In [41]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_8, errors_N_8, resources_N_8 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_26868\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


SLSQP-RealAmplitudes - Depth 1: Average relative error = 0.22240612505083557
SLSQP-RealAmplitudes - Depth 2: Average relative error = 0.0341722620145584
SLSQP-RealAmplitudes - Depth 3: Average relative error = 0.1753041681005697
SLSQP-RealAmplitudes - Depth 4: Average relative error = 0.00013971179185546823
Optimal depth 4 for SLSQP-RealAmplitudes achieved lowest relative error = 2.3306997766340855e-06
Run number with lowest relative error: 7
Relative error at optimal run for SLSQP-RealAmplitudes: 2.3306997766340855e-06
Resource estimates for SLSQP-RealAmplitudes: Gate count = 27, Circuit depth = 17, Function calls = 718

Running VQE for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average relative error = 0.07801950746834627
SLSQP-TwoLocal - Depth 2: Average relative error = 0.03150159908546839
SLSQP-TwoLocal - Depth 3: Average relative error = 0.00012203168142217876
Optimal depth 3 for SLSQP-TwoLocal achieved lowest relative error = 2.8121006838607546e-06
Run numbe

For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 3: Average relative error = 0.00014320687983152396
Optimal depth 3 for SLSQP-EfficientSU2 achieved lowest relative error = 4.935818294724644e-07
Run number with lowest relative error: 5
Relative error at optimal run for SLSQP-EfficientSU2: 4.935818294724644e-07
Resource estimates for SLSQP-EfficientSU2: Gate count = 21, Circuit depth = 13, Function calls = 482

Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


...

P_BFGS-EfficientSU2 - Depth 4: Average relative error = 0.1037414487559832


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Average relative error = 0.23126226862352803


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Average relative error = 3.3273066344450536e-06
Optimal depth 6 for P_BFGS-EfficientSU2 achieved lowest relative error = 2.3354812012511053e-07
Run number with lowest relative error: 10
Relative error at optimal run for P_BFGS-EfficientSU2: 2.3354812012511053e-07
Resource estimates for P_BFGS-EfficientSU2: Gate count = 39, Circuit depth = 25, Function calls = 1496


### Print resource estimates

In [44]:
## Print the min. ansatz depth results
print('The min. ansatz depth results are:')
print(min_depths_N_8)
print()

## Print the relative errors
print('The relative errors are:')
print(errors_N_8)
print()

## Print the resource estimates
print('The resources required are:')
print(resources_N_8)
print()

The min. ansatz depth results are:
{'SLSQP-RealAmplitudes': 4, 'SLSQP-TwoLocal': 3, 'SLSQP-EfficientSU2': 3, 'P_BFGS-RealAmplitudes': 4, 'P_BFGS-TwoLocal': 2, 'P_BFGS-EfficientSU2': 6}

The relative errors are:
{'SLSQP-RealAmplitudes': 2.3306997766340855e-06, 'SLSQP-TwoLocal': 2.8121006838607546e-06, 'SLSQP-EfficientSU2': 4.935818294724644e-07, 'P_BFGS-RealAmplitudes': 3.260549678855917e-08, 'P_BFGS-TwoLocal': 1.20060066981278e-06, 'P_BFGS-EfficientSU2': 2.3354812012511053e-07}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 27, 'circuit_depth': 17, 'function_calls': 718}, 'SLSQP-TwoLocal': {'gate_count': 21, 'circuit_depth': 13, 'function_calls': 637}, 'SLSQP-EfficientSU2': {'gate_count': 21, 'circuit_depth': 13, 'function_calls': 482}, 'P_BFGS-RealAmplitudes': {'gate_count': 27, 'circuit_depth': 17, 'function_calls': 1264}, 'P_BFGS-TwoLocal': {'gate_count': 15, 'circuit_depth': 9, 'function_calls': 2700}, 'P_BFGS-EfficientSU2': {'gate_count': 39, 'circuit_depth':

### For a = 1, b = 0 (N = 16)

### Get the matrix and the zeromode

In [47]:
## Get the matrix and zeromode
nmax = 31
a = 1
gamma = 1
dx = 0.01
c = 0
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 6.75000000e+00 -2.54558441e+01  1.83711731e+01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.54558441e+01  1.29750000e+02 -1.73205081e+02  7.11512474e+01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.83711731e+01 -1.73205081e+02  4.48750000e+02 -4.49132497e+02
   1.53704261e+02  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  7.11512474e+01 -4.49132497e+02  9.63750000e+02
  -8.53097884e+02  2.66223590e+02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000

C:\Users\Owner\AppData\Local\Temp\ipykernel_26868\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The value of <x^2> is:
0.996520371963445



### Estimate resource requirements (fix at 5000 optimization iterations)

In [50]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter)]

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
target_error_threshold = 0.01

# Estimate min. ansatz depths
min_depths_N_16, errors_N_16, resources_N_16 = estimate_resources(matrix, zeromode, optimizers, num_qubits, target_error_threshold, \
                                                               classical_expectation, nmax, L)


Running VQE for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\AppData\Local\Temp\ipykernel_26868\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


SLSQP-RealAmplitudes - Depth 1: Average relative error = 0.1976183110320648
SLSQP-RealAmplitudes - Depth 2: Average relative error = 0.09302120173834794
SLSQP-RealAmplitudes - Depth 3: Average relative error = 0.0637491216681241
SLSQP-RealAmplitudes - Depth 4: Average relative error = 0.032693089120350495
SLSQP-RealAmplitudes - Depth 5: Average relative error = 0.03309424974134166
SLSQP-RealAmplitudes - Depth 6: Average relative error = 0.013061685353905988
SLSQP-RealAmplitudes - Depth 7: Average relative error = 0.0007008506711754932
Optimal depth 7 for SLSQP-RealAmplitudes achieved lowest relative error = 0.00010983100133915506
Run number with lowest relative error: 8
Relative error at optimal run for SLSQP-RealAmplitudes: 0.00010983100133915506
Resource estimates for SLSQP-RealAmplitudes: Gate count = 74, Circuit depth = 37, Function calls = 5014

Running VQE for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average relative error = 0.15067426887628693
SLSQP-TwoLoc

For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 5: Average relative error = 0.0004929820512678852
Optimal depth 5 for SLSQP-EfficientSU2 achieved lowest relative error = 8.028852314103912e-05
Run number with lowest relative error: 9
Relative error at optimal run for SLSQP-EfficientSU2: 8.028852314103912e-05
Resource estimates for SLSQP-EfficientSU2: Gate count = 44, Circuit depth = 26, Function calls = 3015

Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


...

P_BFGS-EfficientSU2 - Depth 6: Average relative error = 0.3308723791485978


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 7: Average relative error = 2.824861170773824


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


KeyboardInterrupt: 

### Next (possible task) --- determine how the entanglement entropy scales with N across different problem instances, given that information about the optimal number of ansatz layers is known. 